In [ ]:
from imageai.Detection import ObjectDetection
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img, save_img
import os
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
def get_detector(model='yolo'):
    execution_path = os.getcwd()
    detector = ObjectDetection()
    detector.setModelTypeAsYOLOv3()
    detector.setModelPath(os.path.join(execution_path , './modules/' + model + '.h5'))
    detector.loadModel()
    custom_objects = detector.CustomObjects(car=True)
    return detector, custom_objects, execution_path
    
def filter_files(files_list, filter_by = None):
    try:
        files_list.remove('.DS_Store')
    except:
        x=1
        
    if filter_by is not None:
        mask = np.logical_not(np.isin(files_list, filter_by))
        result = []
        for f, m in zip(files_list, mask):
            if m == True:
                result.append(f)

        return result
    else:
        return files_list

def get_box(detections):
    area = 0
    points = None
    for detection in detections:
        current_area = (detection['box_points'][2] - detection['box_points'][0]) * (detection['box_points'][3] - detection['box_points'][1])
        if current_area > area:
            area = current_area
            points = detection['box_points']
    return points

def crop_dir(detector, custom_objects, execution_path, source_path, target_path, directory):
    if os.path.exists(target_path + directory) == False:
        os.mkdir(target_path + directory)
    
    filenames = os.listdir(source_path + directory)
    filenames = filter_files(filenames)
    
    for filename in filenames:
        detections = detector.detectCustomObjectsFromImage(
            custom_objects=custom_objects,
            input_image=os.path.join(execution_path , source_path + directory + '/' + filename), 
            minimum_percentage_probability=10,
            extract_detected_objects=False
        )
        box = get_box(detections)
        img = load_img(source_path + directory + '/' + filename)
        cropped_img = img.crop(box)
        save_img(target_path + directory + '/' + filename, cropped_img)
    
def crop(source_path, target_path, dirs_path, model='yolo'):
    dirs = os.listdir(dirs_path)
    target_dirs = os.listdir(target_path)
    dirs = filter_files(dirs, target_dirs)

    if os.path.exists(target_path) == False:
        os.mkdir(target_path)
        
    detector, custom_objects, execution_path = get_detector(model)
    
    for i, directory in enumerate(dirs):
        print('Directory', str(i), 'of', str(len(dirs)), '...')
        crop_dir(detector, custom_objects, execution_path, source_path, target_path, directory)
        print('Images in directory', str(i), '"' + directory + '"', 'cropped')

In [ ]:
crop(
    './stanford-car-dataset-by-classes-folder/car_data/', 
    './stanford-car-dataset-by-classes-folder/cropped_car_data/',
    './stanford-car-dataset-by-classes-folder/train_dataset/',
    'yolo',
)

## Preview

In [ ]:
execution_path = os.getcwd()

detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(os.path.join(execution_path , "./modules/yolo.h5"))
detector.loadModel()
custom_objects = detector.CustomObjects(car=True)

detections = detector.detectCustomObjectsFromImage(
    custom_objects=custom_objects,
    input_image=os.path.join(execution_path , "./stanford-car-dataset-by-classes-folder/car_data/Acura Integra_Type_R Coupe 2001/04795.jpg"), 
    minimum_percentage_probability=30,
    #output_image_path=os.path.join(execution_path , "image2new.jpg"),
    #extract_detected_objects=True
)

print(isinstance(detections, tuple))
print(detections)
print(detections)

img = load_img('./stanford-car-dataset-by-classes-folder/car_data/Acura Integra_Type_R Coupe 2001/04795.jpg')
print(img.size)
w,h = img.size

crop_img = img.crop(get_box(detections))
plt.imshow(crop_img)